<a href="https://colab.research.google.com/github/lilacs2039/VoicePix2Pix/blob/tmp/Usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境構築

## 入力ファイル・出力ファイルのディレクトリを指定するファイルを作成
__セル内を適宜かきかえること__

In [0]:
with open('.env',"w") as f:
    f.write("dataRootDir = path/to/inputData\r\n")
    f.write("outputDir = ./output")


In [0]:
import os
d = []
for  line in open('.env'):
    if not line.__contains__('=') : continue
    d.append(line.strip().split(' = '))
d = dict(d)
os.environ['dataRootDir'] = d['dataRootDir']
dataRootDir = d['dataRootDir']
os.environ['outputDir'] = d['outputDir']
outputDir = d['outputDir']
print(d)

#world4py インストール
%cd /content
!git clone --depth 1 https://github.com/yamachu/WORLD4py
%cd /content/WORLD4py
!python setup.py install
%cd /content

!pip install -r requirements.txt


{'dataRootDir': "'F:/Users/Chilly/bulk/dataset/facades'", 'outputDir': "'./output'"}


distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


# 学習開始

In [0]:
print(os.environ['dataRootDir'])
!echo $dataRootDir

'F:/Users/Chilly/bulk/dataset/facades'


$dataRootDir


In [0]:
#GPUありで学習
cmd  = "python train_facade.py -g 0 -i {} --out {} --snapshot_interval 10000".format(d['dataRootDir'],d['outputDir'])
import subprocess
res = subprocess.call(cmd)
print(res)

1


In [0]:
#GPUありで学習
!python train_facade.py -g 0 -i $dataRootDir --out $outputDir --snapshot_interval 10000


GPU: 0
# Minibatch-size: 1
# epoch: 200

load dataset start
    from: 'F:/Users/Chilly/bulk/dataset/facades'
    range: [1, 300)


C:\Users\Chilly.Ice-desktop\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Traceback (most recent call last):
  File "train_facade.py", line 119, in <module>
    main()
  File "train_facade.py", line 70, in main
    train_d = FacadeDataset(args.dataset, data_range=(1,300))
  File "E:\Users\Chilly\My Documents\GoogleBackup\pycharm\chainer-pix2pix\facade_dataset.py", line 28, in __init__
    img = Image.open(dataDir+"/cmp_b%04d.jpg"%i)
  File "C:\Users\Chilly.Ice-desktop\Anaconda3\lib\site-packages\PIL\Image.py", line 2548, in open
    fp = builtins.open(filename, "rb")
OSError: [Errno 22] Invalid argument: "'F:/Users/Chilly/bulk/dataset/facades'/cmp_b0001.jpg"
